# Inspect the phase angles and reduced magnitudes for DP1 objects
Author: James E. Robinson

Query all objects and observations in DP1 and generate summary statistics for the phase angles and reduced magnitudes: range, min, max, mean.
This should give an indication of objects that may have lightcurve amplitude variations or interesting colours.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.units as u

from lsst.rsp import get_tap_service

In [2]:
service = get_tap_service("tap")
assert service is not None

# Query for all properties from the SSObject and and MPCORB tables

In [3]:
query = """SELECT * 
            FROM
                dp1.MPCORB as mpc
                INNER JOIN dp1.SSObject as sso
                ON mpc.ssObjectId = sso.ssObjectId
                """

In [4]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

Job phase is COMPLETED


In [5]:
assert job.phase == 'COMPLETED'
result = job.fetch_result()
print(len(result))

431


In [6]:
df_obj = pd.DataFrame(result)

In [7]:
# calculate the semimajor axis
df_obj["a"] = df_obj["q"] / (1.0 - df_obj["e"])

In [8]:
# df_obj

# Query for all observations in the DiaSource and SSSource tables

In [9]:
query = """
SELECT *
FROM
    dp1.DiaSource as dia
INNER JOIN
    dp1.SSSource as sss
ON
    dia.diaSourceId = sss.diaSourceId
"""

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()

In [ ]:
assert job.phase == 'COMPLETED'
result = job.fetch_result()
print(len(result))

In [ ]:
df_obs = pd.DataFrame(result)

In [ ]:
# df_obs

# Calculate the magnitudes from flux
See "Using astropy for unit conversion" https://community.lsst.org/t/photocalib-has-replaced-calib-welcoming-our-nanojansky-overlords/3648

In [ ]:
# Flux in nanoJanskys to AB magnitudes - NB negative fluxes will give nans
df_obs["apABmag"] = (np.array(df_obs["apFlux"]) * u.nJy).to(u.ABmag).value
df_obs["psfABmag"] = (np.array(df_obs["psfFlux"]) * u.nJy).to(u.ABmag).value
df_obs["trailABmag"] = (np.array(df_obs["trailFlux"]) * u.nJy).to(u.ABmag).value

# Calculate the reduced magnitude using the distances of each observation
thdist = df_obs['topocentricDist']*df_obs['heliocentricDist']
df_obs["red_apABmag"] = df_obs['apABmag'] - 5.0*np.log10(thdist)
df_obs["red_psfABmag"] = df_obs['psfABmag'] - 5.0*np.log10(thdist)
df_obs["red_trailABmag"] = df_obs['trailABmag'] - 5.0*np.log10(thdist)

In [ ]:
# df_obs

# Group all detections by ssObjectId
This will allow us to see how the overall phase angle coverage and reduced magnitude changes per object (across all bands)

In [ ]:
# gb_obs = df_obs.groupby("ssObjectId")
gb_obs = df_obs.dropna(subset = ["red_apABmag","red_psfABmag"]).groupby("ssObjectId") # we need to drop nan mags otherwise the summary stat for np.ptp will also be nan

In [ ]:
# range
df_ptp = gb_obs[['red_apABmag',"red_psfABmag","phaseAngle"]].agg(np.ptp).reset_index()
df_ptp = df_ptp.rename({'red_apABmag':"range_red_apABmag",
                        'red_psfABmag':"range_red_psfABmag",
                        "phaseAngle":"range_phaseAngle"}, axis = 1)
df_ptp

In [ ]:
# min
df_min = gb_obs[["red_apABmag","red_psfABmag","phaseAngle"]].min().reset_index()
df_min = df_min.rename({'red_apABmag':"min_red_apABmag",
                        'red_psfABmag':"min_red_psfABmag",
                        "phaseAngle":"min_phaseAngle"}, axis = 1)
df_min

In [ ]:
# max
df_max = gb_obs[["red_apABmag","red_psfABmag","phaseAngle"]].max().reset_index()
df_max = df_max.rename({'red_apABmag':"max_red_apABmag",
                        'red_psfABmag':"max_red_psfABmag",
                        "phaseAngle":"max_phaseAngle"}, axis = 1)
df_max

In [ ]:
# mean
df_mean = gb_obs[["red_apABmag","red_psfABmag","phaseAngle"]].mean().reset_index()
df_mean = df_mean.rename({'red_apABmag':"mean_red_apABmag",
                          'red_psfABmag':"mean_red_psfABmag",
                        "phaseAngle":"mean_phaseAngle"}, axis = 1)

In [ ]:
# Overall summary dataframe
df_mag = df_ptp.copy()
df_mag = df_mag.merge(df_min, on = "ssObjectId")
df_mag = df_mag.merge(df_max, on = "ssObjectId")
df_mag = df_mag.merge(df_mean, on = "ssObjectId")
df_mag = df_mag.merge(df_obj[["ssObjectId","a"]], on = "ssObjectId")

df_mag

In [ ]:
df_mag.sort_values("range_red_apABmag")

In [ ]:
df_plot = df_mag

for x_plot in ["min_phaseAngle","max_phaseAngle","range_phaseAngle"]:
               
    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0,0])
    
    ax1.hist(df_plot[x_plot])

    med_x = np.median(df_plot[x_plot])
    print("median({}) = {}".format(x_plot,med_x))
    ax1.axvline(med_x, c= "r", label = "{:.2f}".format(med_x))
        
    ax1.legend()
    ax1.set_xlabel(x_plot)
    ax1.set_ylabel("number")
    
    plt.show()

In [ ]:
# plot the maximum observed phase angle for each object as a function of semimajor axis
x_plot = "a"
y_plot = "max_phaseAngle"
c_plot = "range_phaseAngle"
df_plot = df_mag.sort_values(c_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

s1 = ax1.scatter(df_plot[x_plot],df_plot[y_plot], c = df_plot[c_plot])
cbar1 = plt.colorbar(s1)

# Maximum possible phase angle as a function of semimajor axis (for circular orbit)
# _x = np.linspace(np.amin(df_plot[x_plot]),np.amax(df_plot[x_plot]))
# ax1.plot(_x, np.arcsin(1.0/_x), c = "r")

ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
cbar1.set_label(c_plot)

plt.show()


In [ ]:
# plot the object with the largest phase curve coverage
ssobjid = df_mag.iloc[np.argmax(df_mag["range_phaseAngle"])]["ssObjectId"]

In [ ]:
# Plot the phase curve
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
print(ssobjid,mpc_des)

x_plot = "phaseAngle"
y_plot = "red_apABmag"
df_plot = df_obs[df_obs["ssObjectId"] == ssobjid].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))
    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                edgecolor = "C{}".format(i),
                facecolor = "none",
                label = filt)
    ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
             c = "C{}".format(i),)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()

plt.title("{}\n{}".format(ssobjid,mpc_des))

plt.show()


In [ ]:
# Look at the lightcurve (note that this is very similar to the phase curve given the small observing baseline)
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
print(ssobjid,mpc_des)

x_plot = "midpointMjdTai"
y_plot = "red_apABmag"
df_plot = df_obs[df_obs["ssObjectId"] == ssobjid].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))
    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                edgecolor = "C{}".format(i),
                facecolor = "none",
                label = filt)
    ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
             c = "C{}".format(i),)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()

plt.title("{}\n{}".format(ssobjid,mpc_des))

plt.show()


# Unusual photometry?
ssObjectId = 21163611662530641 appears to have extremely faint detections in the z band. This seems fishy, let's look at the observations:

In [ ]:
# # Inspect all observations in full
# df = df_obs[df_obs["ssObjectId"] == ssobjid]
# # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# #     display(df)
# df

The faint detections have a number of nan values, which appear to be caused by an attempted dipole fit (`dipoleFitAttempted == True`)
See also ssObjectId for another example 21163611677341266.

Furthermore some objects (e.g. ssObjectId = 23133931615302707) still have faint measurements, and these have nan values for trailing measurements (`shape_flag == True`).
NB not all observations with `shape_flag == True` appear to be outlying.

In [ ]:
# dipoleFitAttempted==True example objects
# ssobjid = 21163611662530641
# ssobjid = 21163611677341266

# shape_flag==True example object
ssobjid = 23133931615302707

In [ ]:
# # Inspect all observations in full
# df = df_obs[df_obs["ssObjectId"] == ssobjid]
# # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# #     display(df)
# df

In [ ]:
# Plot the phase curve
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
print(ssobjid,mpc_des)

x_plot = "phaseAngle"
y_plot = "red_apABmag"
df_plot = df_obs[df_obs["ssObjectId"] == ssobjid].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))
    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                edgecolor = "C{}".format(i),
                facecolor = "none",
                label = filt)
    ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
             c = "C{}".format(i),)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()

plt.title("{}\n{}".format(ssobjid,mpc_des))

plt.show()


In [ ]:
df_obs[~(df_obs["dipoleFitAttempted"])].dropna(subset = ["red_apABmag"])

# Group by ssObjectId and band
These summary stats allow us to assess changes in brightness in each band for each object.
Significant changes in a single band could indicate a rotational amplitude (or weird measurements).
Changes between mean brightness in different bands for a given object indicate colours.

In [ ]:
# We want to exclude unusual photometry from our summary statistics so we define a mask
all_obs_mask = ~((df_obs["dipoleFitAttempted"]) | (df_obs["shape_flag"]))

# drop anything with a flag for dipole or shape
df_obs[~all_obs_mask][["shape_flag","dipoleFitAttempted"]]

In [ ]:
# generate the summary statistics, by object and band, and excluding possible weird photometry
gb_obs_band = df_obs[all_obs_mask].dropna(subset = ["red_apABmag","red_psfABmag","red_trailABmag"]).groupby(["ssObjectId","band"])

df_ptp_band = gb_obs_band[['red_apABmag',"red_psfABmag","red_trailABmag","phaseAngle"]].agg(np.ptp).reset_index()
df_ptp_band = df_ptp_band.rename({'red_apABmag':"range_red_apABmag",
                                  'red_psfABmag':"range_red_psfABmag",
                                  'red_trailABmag':"range_red_trailABmag",
                        "phaseAngle":"range_phaseAngle"}, axis = 1)

df_min_band = gb_obs_band[["red_apABmag","red_psfABmag","red_trailABmag","phaseAngle"]].min().reset_index()
df_min_band = df_min_band.rename({'red_apABmag':"min_red_apABmag",
                                  'red_psfABmag':"min_red_psfABmag",
                                  'red_trailABmag':"min_red_trailABmag",
                        "phaseAngle":"min_phaseAngle"}, axis = 1)

df_max_band = gb_obs_band[["red_apABmag","red_psfABmag","red_trailABmag","phaseAngle"]].min().reset_index()
df_max_band = df_max_band.rename({'red_apABmag':"max_red_apABmag",
                                  'red_psfABmag':"max_red_psfABmag",
                                  'red_trailABmag':"max_red_trailABmag",
                        "phaseAngle":"max_phaseAngle"}, axis = 1)

df_mean_band = gb_obs_band[["red_apABmag","red_psfABmag","red_trailABmag","phaseAngle"]].mean().reset_index()
df_mean_band = df_mean_band.rename({'red_apABmag':"mean_red_apABmag",
                                    'red_psfABmag':"mean_red_psfABmag",
                                    'red_trailABmag':"mean_red_trailABmag",
                        "phaseAngle":"mean_phaseAngle"}, axis = 1)

df_mag_band = df_ptp_band.copy()
df_mag_band = df_mag_band.merge(df_min_band, on = ["ssObjectId","band"])
df_mag_band = df_mag_band.merge(df_max_band, on = ["ssObjectId","band"])
df_mag_band = df_mag_band.merge(df_mean_band, on = ["ssObjectId","band"])
# df_mag_band = df_mag_band.merge(df_obj, on = "ssObjectId")

# Inspect the object with the most observations

In [ ]:
# df_obj.iloc[np.argmax(df_obj["numObs"])]

In [ ]:
ssobjid = df_obj.iloc[np.argmax(df_obj["numObs"])]["ssObjectId"] # ssObjectId with the most observations

In [ ]:
# df = df_obs[(all_obs_mask) & (df_obs["ssObjectId"] == ssobjid)]
# # with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
# #     display(df)
# df

In [ ]:
# df_obs[(~all_obs_mask) & (df_obs["ssObjectId"] == ssobjid)]

In [ ]:
# plot the phase curve and indicate observations that we have excluded
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
x_plot = "phaseAngle"
y_plot = "red_apABmag"
df_plot = df_obs[all_obs_mask & (df_obs["ssObjectId"] == ssobjid)].sort_values(x_plot)
df_plot2 = df_obs[(~all_obs_mask) & (df_obs["ssObjectId"] == ssobjid)].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

            
mlist = ["o","s","*"]
slist = [50,75,100]
for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))
    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                edgecolor = "C{}".format(i),
                facecolor = "none",
                # marker = mlist[i],
                # s = slist[i],
                label = filt)
    ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
             c = "C{}".format(i),)

ax1.scatter(df_plot2[x_plot],df_plot2[y_plot],
            s = 100,
            marker = "x", c = "k", label = "cut", zorder = 5)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)
ax1.invert_yaxis()

plt.title("{}\n{}".format(ssobjid,mpc_des))
plt.savefig("{}_{}".format(ssobjid,"_".join(mpc_des.split(" "))))

plt.show()


# What is the relation between the different flux/mag measurements?
DiaSource gives a number of flux measurements, in particular:
- aperture
- psf
- trailed

In [ ]:
# Plot the phase curve in multiple flux methods for a given object
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
print(ssobjid,mpc_des)

x_plot = "phaseAngle"
df_plot = df_obs[df_obs["ssObjectId"] == ssobjid].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

for y_plot,m,ls in zip(["red_apABmag","red_psfABmag","red_trailABmag"],["o","s","^"],["-",":","--"]):
    for i,filt in enumerate(np.unique(df_plot["band"])):
        _df_plot = df_plot[df_plot["band"]==filt]
        ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                    edgecolor = "C{}".format(i),
                    facecolor = "none",
                    marker = m,
                    label = "{} {}".format(y_plot,filt))
        ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
                 ls = ls,
                 c = "C{}".format(i),)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel("mag")
ax1.invert_yaxis()

plt.title("{}\n{}".format(ssobjid,mpc_des))

plt.show()


In [ ]:
# check the relation between different flux methods
x_plot = "red_psfABmag"
y_plot = "red_apABmag"
y_plot2 = "red_trailABmag"
df_plot = df_obs

for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 2)
    ax1 = plt.subplot(gs[0,0])
    ax2 = plt.subplot(gs[0,1])

    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                c = "C{}".format(i),)

    ax1.plot([np.amin(_df_plot[x_plot]),np.amax(_df_plot[x_plot])],
              [np.amin(_df_plot[x_plot]),np.amax(_df_plot[x_plot])],
                                               c = "k")

    ax2.scatter(_df_plot[x_plot],_df_plot[y_plot2],
                c = "C{}".format(i),)

    ax2.plot([np.amin(_df_plot[x_plot]),np.amax(_df_plot[x_plot])],
              [np.amin(_df_plot[x_plot]),np.amax(_df_plot[x_plot])],
                                               c = "k")

    fig.suptitle(filt)
    ax1.set_xlabel(x_plot)
    ax1.set_ylabel(y_plot)
    ax2.set_xlabel(x_plot)
    ax2.set_ylabel(y_plot2)

    plt.tight_layout()
    plt.show()


In [ ]:
# What method should we use to assess changes in brightness of an object?
# assess the relation between the range statistics of the reduced magnitudes
x_plot = "range_red_psfABmag"
df_plot = df_mag_band

for y_plot in ["range_red_apABmag","range_red_trailABmag"]:
               
    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1)
    ax1 = plt.subplot(gs[0,0])
    
    ax1.scatter(df_plot[x_plot],df_plot[y_plot])
        

    ax1.plot([np.amin(df_plot[x_plot]),np.amax(df_plot[x_plot])],
              [np.amin(df_plot[x_plot]),np.amax(df_plot[x_plot])],
                                               c = "k")
    
    ax1.set_xlabel(x_plot)
    ax1.set_ylabel(y_plot)
    
    plt.show()

Aperture magnitudes appear to overestimate the range statistic, use psf or trailed instead

In [ ]:
df_mag_band.sort_values("range_red_psfABmag")

In [ ]:
ssobjid = df_mag_band.iloc[np.argmax(df_mag_band["range_red_psfABmag"])]["ssObjectId"]

In [ ]:
# plot the phase curve and indicate observations that we have excluded
mpc_des = df_obj[df_obj["ssObjectId"]==ssobjid].iloc[0]["mpcDesignation"]
x_plot = "phaseAngle"
y_plot = "red_apABmag"
df_plot = df_obs[all_obs_mask & (df_obs["ssObjectId"] == ssobjid)].sort_values(x_plot)
df_plot2 = df_obs[(~all_obs_mask) & (df_obs["ssObjectId"] == ssobjid)].sort_values(x_plot)

fig = plt.figure()
gs = gridspec.GridSpec(1, 1)
ax1 = plt.subplot(gs[0,0])

            
mlist = ["o","s","*"]
slist = [50,75,100]
for i,filt in enumerate(np.unique(df_plot["band"])):
    _df_plot = df_plot[df_plot["band"]==filt]
    print(filt,len(_df_plot))
    ax1.scatter(_df_plot[x_plot],_df_plot[y_plot],
                edgecolor = "C{}".format(i),
                facecolor = "none",
                # marker = mlist[i],
                # s = slist[i],
                label = filt)
    ax1.plot(_df_plot[x_plot],_df_plot[y_plot],
             c = "C{}".format(i),)

ax1.scatter(df_plot2[x_plot],df_plot2[y_plot],
            s = 100,
            marker = "x", c = "k", label = "cut", zorder = 5)

ax1.legend()
ax1.set_xlabel(x_plot)
ax1.set_ylabel(y_plot)

plt.title("{}\n{}".format(ssobjid,mpc_des))

plt.show()
